In [10]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Load the data
file_path = '/Users/leyuezhang/Desktop/Copy_book2.csv'
data = pd.read_csv(file_path)

# Remove the last column 'Unnamed: 123' which appears to be an empty column or irrelevant
data = data.drop(columns=['Unnamed: 123'])

# Melt the data to long format
data_long = data.melt(id_vars=['type'], var_name='Date', value_name='Mobility_Count')

# Convert the Date column to datetime, while handling the inconsistent date format
data_long['Date'] = pd.to_datetime(data_long['Date'], format='%m/%d/%y', errors='coerce')

# Remove rows with NaT in Date column
data_long = data_long.dropna(subset=['Date'])

# Define flood periods
data_long['Flood_Period'] = 'benchmark'
data_long.loc[(data_long['Date'] >= '2020-06-13') & (data_long['Date'] <= '2020-06-16'), 'Flood_Period'] = 'pre_flood'
data_long.loc[(data_long['Date'] >= '2020-06-17') & (data_long['Date'] <= '2020-07-08'), 'Flood_Period'] = 'flood'
data_long.loc[(data_long['Date'] >= '2020-07-10') & (data_long['Date'] <= '2020-07-23'), 'Flood_Period'] = 'post_flood'

# Create dummy variables
data_long['PreFlood'] = np.where(data_long['Flood_Period'] == 'pre_flood', 1, 0)
data_long['Flood'] = np.where(data_long['Flood_Period'] == 'flood', 1, 0)
data_long['PostFlood'] = np.where(data_long['Flood_Period'] == 'post_flood', 1, 0)

# Log-transform the Mobility_Count
data_long['Log_Mobility'] = np.log(data_long['Mobility_Count'] + 1)  # Adding 1 to avoid log(0)

# Function to run the regression for each POI type and format the results including p-values
def run_regression_for_poi_with_pvalues(poi_type):
    poi_data = data_long[data_long['type'] == poi_type]
    model = smf.ols(formula='Log_Mobility ~ PreFlood + Flood + PostFlood', data=poi_data)
    results = model.fit()
    
    summary_table = {
        'POI_Type': poi_type,
        'Treat*PreFlood Coefficient': results.params['PreFlood'],
        'Treat*PreFlood Std. Error': results.bse['PreFlood'],
        'Treat*PreFlood p-value': results.pvalues['PreFlood'],
        'Treat*Flood Coefficient': results.params['Flood'],
        'Treat*Flood Std. Error': results.bse['Flood'],
        'Treat*Flood p-value': results.pvalues['Flood'],
        'Treat*PostFlood Coefficient': results.params['PostFlood'],
        'Treat*PostFlood Std. Error': results.bse['PostFlood'],
        'Treat*PostFlood p-value': results.pvalues['PostFlood'],
        'Observations': int(results.nobs),
        'R-squared': results.rsquared
    }
    return summary_table

# List to store results
results_list = []

# Run the regression for each POI type
for poi_type in data['type'].unique():
    summary = run_regression_for_poi_with_pvalues(poi_type)
    results_list.append(summary)

# Convert the results list to a DataFrame
final_results_with_pvalues = pd.DataFrame(results_list)

# Print the final results
print(final_results_with_pvalues)


                    POI_Type  Treat*PreFlood Coefficient  \
0              Accommodation                   -0.079432   
1                  Education                   -0.298452   
2       Healthcare & Medical                   -0.223987   
3   Financial & Professional                   -0.311375   
4                  Religious                   -0.148200   
5                Residential                   -0.243343   
6             Transportation                   -0.265994   
7           Energy & Storage                   -0.145792   
8         Outdoor Recreation                    0.229899   
9            Public agencies                   -0.488325   
10         Retail & Shopping                   -0.333869   
11             Entertainment                   -0.337298   

    Treat*PreFlood Std. Error  Treat*PreFlood p-value  \
0                    0.151662                0.601439   
1                    0.137648                0.032149   
2                    0.180247                0.2

In [2]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

# Load the data
file_path = '/Users/leyuezhang/Desktop/Copy_book2.csv'
data = pd.read_csv(file_path)

# Remove the last column 'Unnamed: 123' which appears to be an empty column or irrelevant
data = data.drop(columns=['Unnamed: 123'])

# Melt the data to long format
data_long = data.melt(id_vars=['type'], var_name='Date', value_name='Mobility_Count')

# Convert the Date column to datetime, while handling the inconsistent date format
data_long['Date'] = pd.to_datetime(data_long['Date'], format='%m/%d/%y', errors='coerce')

# Remove rows with NaT in Date column
data_long = data_long.dropna(subset=['Date'])

# Define flood periods
data_long['Flood_Period'] = 'benchmark'
data_long.loc[(data_long['Date'] >= '2020-06-13') & (data_long['Date'] <= '2020-06-16'), 'Flood_Period'] = 'pre_flood'
data_long.loc[(data_long['Date'] >= '2020-06-17') & (data_long['Date'] <= '2020-07-08'), 'Flood_Period'] = 'flood'
data_long.loc[(data_long['Date'] >= '2020-07-10') & (data_long['Date'] <= '2020-07-23'), 'Flood_Period'] = 'post_flood'

# Create dummy variables
data_long['PreFlood'] = np.where(data_long['Flood_Period'] == 'pre_flood', 1, 0)
data_long['Flood'] = np.where(data_long['Flood_Period'] == 'flood', 1, 0)
data_long['PostFlood'] = np.where(data_long['Flood_Period'] == 'post_flood', 1, 0)

# Log-transform the Mobility_Count
data_long['Log_Mobility'] = np.log(data_long['Mobility_Count'] + 1)  # Adding 1 to avoid log(0)

# Function to run the regression for each POI type and format the results including p-values and percentage changes
def run_regression_for_poi_with_pvalues(poi_type):
    poi_data = data_long[data_long['type'] == poi_type]
    model = smf.ols(formula='Log_Mobility ~ PreFlood + Flood + PostFlood', data=poi_data)
    results = model.fit()
    
    def calculate_percentage_change(coefficient):
        if coefficient > 0:
            return (np.exp(coefficient) - 1) * 100
        else:
            return (1 - np.exp(coefficient)) * 100

    def format_coefficient_with_significance(coefficient, p_value):
        if p_value < 0.01:
            return f"{coefficient:.4f}***"
        elif 0.01 <= p_value < 0.05:
            return f"{coefficient:.4f}**"
        elif 0.05 <= p_value < 0.1:
            return f"{coefficient:.4f}*"
        else:
            return f"{coefficient:.4f}"

    summary_table = {
        'POI_Type': poi_type,
        'Treat*PreFlood Coefficient': format_coefficient_with_significance(results.params['PreFlood'], results.pvalues['PreFlood']),
        'Treat*PreFlood Std. Error': results.bse['PreFlood'],
        'Treat*PreFlood p-value': results.pvalues['PreFlood'],
        'Treat*PreFlood Percentage Change (%)': calculate_percentage_change(results.params['PreFlood']) if results.params['PreFlood'] > 0 else -calculate_percentage_change(results.params['PreFlood']),
        'Treat*Flood Coefficient': format_coefficient_with_significance(results.params['Flood'], results.pvalues['Flood']),
        'Treat*Flood Std. Error': results.bse['Flood'],
        'Treat*Flood p-value': results.pvalues['Flood'],
        'Treat*Flood Percentage Change (%)': calculate_percentage_change(results.params['Flood']) if results.params['Flood'] > 0 else -calculate_percentage_change(results.params['Flood']),
        'Treat*PostFlood Coefficient': format_coefficient_with_significance(results.params['PostFlood'], results.pvalues['PostFlood']),
        'Treat*PostFlood Std. Error': results.bse['PostFlood'],
        'Treat*PostFlood p-value': results.pvalues['PostFlood'],
        'Treat*PostFlood Percentage Change (%)': calculate_percentage_change(results.params['PostFlood']) if results.params['PostFlood'] > 0 else -calculate_percentage_change(results.params['PostFlood']),
        'Observations': int(results.nobs),
        'R-squared': results.rsquared
    }
    return summary_table

# List to store results
results_list = []

# Run the regression for each POI type
for poi_type in data['type'].unique():
    summary = run_regression_for_poi_with_pvalues(poi_type)
    results_list.append(summary)

# Convert the results list to a DataFrame
final_results_with_pvalues = pd.DataFrame(results_list)

# Save the final results to a CSV file
output_file_path = '/Users/leyuezhang/Desktop/final_results_with_pvalues_and_percentages.csv'
final_results_with_pvalues.to_csv(output_file_path, index=False)

# Print the final results
print(final_results_with_pvalues)


                    POI_Type Treat*PreFlood Coefficient  \
0              Accommodation                    -0.0794   
1                  Education                  -0.2985**   
2       Healthcare & Medical                    -0.2240   
3   Financial & Professional                    -0.3114   
4                  Religious                    -0.1482   
5                Residential                  -0.2433**   
6             Transportation                    -0.2660   
7           Energy & Storage                    -0.1458   
8         Outdoor Recreation                     0.2299   
9            Public agencies                    -0.4883   
10         Retail & Shopping                  -0.3339**   
11             Entertainment                   -0.3373*   

    Treat*PreFlood Std. Error  Treat*PreFlood p-value  \
0                    0.151662                0.601439   
1                    0.137648                0.032149   
2                    0.180247                0.216454   
3   